# Pytorch Introduction

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/m12sl/dl-hse-2020/blob/master/03-common-things/vanishing%20gradients%20and%20cnn.ipynb)

В этой тетрадке мы напишем познакомимся с основными операциями для построения сверточных сетей.

**Цели тетрадки**

1. Знакомство со свертками и пуллингами
1. Попрактиковаться в построении сетей


**План работы**

1. Поэкспериментировать с затуханием градиентов
1. Познакомиться со сверточными сетями и операциями для их построения


## (повтор) Материалы по pytorch:

+ https://pytorch.org/resources/
+ https://pytorch.org/docs/stable/index.html
+ ходить по исходникам с помощью IDE
+ [Классная статья про pytorch internal](http://blog.ezyang.com/2019/05/pytorch-internals/)

In [ ]:
# install requirements
! pip install torchviz torchvision

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm_notebook as tqdm
from collections import defaultdict

from IPython.display import clear_output

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

from torchvision import datasets, transforms
from torchvision.datasets import FashionMNIST
from torchvision import transforms

**Если вы пользуетесь Colab, проверьте, что вам доступен GPU (иначе включите в настройках GPU-acceleration)**

In [ ]:
torch.cuda.is_available()

Для экспериментов предлагается снова взять **FashionMNIST**.

Мы начнем с моделей на полносвязных слоях, так что нужно преобразовать картинки в вектора:

In [ ]:
transform_to_vector = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Lambda(lambda t: t.reshape(-1)),
])

train_dataset = FashionMNIST("./tmp", train=True, download=True, transform=transform_to_vector)
val_dataset = FashionMNIST("./tmp", train=False, download=True, transform=transform_to_vector)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=32)

plt.figure(figsize=[6, 6])
for i in range(4):
    plt.subplot(2, 2, i + 1)
    plt.title("Label: %i" % train_dataset[i][1])
    plt.imshow(train_dataset[i][0].reshape(28, 28), cmap='gray')  # don't forget convert vector to image

У pytorch моделей есть множество встроенных хелперов.
Например генераторы для обхода параметров: `model.named_parameters()` и `model.parameters()`

In [ ]:
model = nn.Sequential(
    nn.Linear(7, 11),
    nn.Sigmoid(),
    nn.Linear(11, 10),
)

for name, p in model.named_parameters():
    print(name, p.shape)

**(0.05 балла)** Напишите функцию для подсчета количества обучаемых параметров:

In [ ]:
def count_parameters(model):
    <your code>
    return num

model = nn.Sequential(
    nn.Linear(7, 11),
    nn.Sigmoid(),
    nn.Linear(11, 10),
)

assert count_parameters(model) == 208

**(0.05 балла)** Напишите функцию для вычисления ($L_2$) норм градиентов на каждый из параметров.

**NB: функция должна работать на CPU и на GPU.**

In [ ]:
def get_grad_norms(model):
    <your code>
    return {"some.weight": some float}


model = nn.Sequential(
    nn.Linear(7, 11),
    nn.Sigmoid(),
    nn.Linear(11, 10),
)

x = torch.ones(13, 7)
loss = model(x).mean()
loss.backward()

assert get_grad_norms(model).keys() == {"0.weight", "0.bias", "2.weight", "2.bias"}

if torch.cuda.is_available():
    device = "cuda"
    model.to(device)
    x = x.to(device)
    loss = model(x).mean()
    loss.backward()
    assert get_grad_norms(model).keys() == {"0.weight", "0.bias", "2.weight", "2.bias"}
    print("All is fine")
else:
    print("GPU unchecked")

**(0.2 балла)** Допишите тренировочный цикл так, чтобы кроме метрик логгировались и выводились еще и нормы градиентов на тренировочных шагах

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
def train(model, optimizer, dataloader): 
    model.to(device)
    model.train()
    logs = defaultdict(list)
    for x, y in tqdm(dataloader):
        <your code here>
        
        # информацию про нормы градиентов каждого слоя сложите как какой-нибудь 
        # logs['grad_something']
        # у вас должно быть две скалярных переменных: с метрикой и лоссом
        logs['acc'].append(acc.item())
        logs['loss'].append(loss.item())
    return logs

def validate(model, dataloader):
    model.to(device)
    model.eval()
    logs = defaultdict(list)
    for x, y in tqdm(dataloader):
        <your code>
        # здесь подсчет градиентов не нужен
        # у вас должно быть две скалярных переменных: с метрикой и лоссом
        logs['acc'].append(acc.item())
        logs['loss'].append(loss.item())
    
    return {k: [np.mean(v)] for k, v in logs.items()}

def plot_logs(logs):
    clear_output()
    plt.figure()
    plt.plot(logs['acc'], zorder=1)
    plt.scatter(logs['steps'], logs['val_acc'], marker='+', s=180, c='orange', label='val', zorder=2)
    plt.show()

    plt.figure()
    # добавьте отображение градиентов здесь
    # для отображения подписей воспользуйтесь label&legend
    # plt.plot(..., label=name)
    # plt.legend() 
    <your code>        
    plt.legend()
    plt.grid()
    plt.show()


def train_model(model, optimizer, train_loader, val_loader, epochs=10):
    logs = defaultdict(list)
    for epoch in range(epochs):
        train_logs = train(model, opt, train_loader)
        
        # вы вольны переписать объединение логов
        for k, v in train_logs.items():
            logs[k].extend(v)

        val_logs = validate(model, val_loader)
        for k, v in val_logs.items():
            logs[f'val_{k}'].extend(v)
        logs['steps'].append(len(logs['loss']))

        clear_output()
        plot_logs(logs)

**(0.1 балл)** Давайте сравним кривые обучения и нормы градиентов для двух сетей:

- densenet из прошлого семинара + sigmoids
- densenet + relu

**(+дополнительный 0.1 балл)** Можно ли что-то сделать, чтобы заставить densenet_sigmoid обучаться лучше (без изменения функций активаций и BN)?

In [ ]:
densenet_sigmoid = nn.Sequential(
    nn.Linear(784, 40),
    nn.Sigmoid(),
    nn.Linear(40, 40),
    nn.Sigmoid(),
    nn.Linear(40, 10),
    nn.LogSoftmax(dim=-1),
)

opt = torch.optim.SGD(densenet_sigmoid.parameters(), lr=0.01)
train_model(densenet_sigmoid, opt, train_loader, val_loader)

In [ ]:
densenet_relu = nn.Sequential(
    nn.Linear(784, 40),
    nn.ReLU(),
    nn.Linear(40, 40),
    nn.ReLU(),
    nn.Linear(40, 10),
    nn.LogSoftmax(dim=-1),
)

opt = torch.optim.SGD(densenet_relu.parameters(), lr=0.01)
train_model(densenet_relu, opt, train_loader, val_loader)

## Vanishing Gradients

**(0.2 балла)** Предлагается сделать и проучить глубокую полносвязную сеть (10 линейных слоев, по 20 юнитов в каждом, перемежаемых нелинейностями) в нескольких вариациях:

- Densenet + Sigmoid
- Densenet + ReLU
- DenseResNet + Sigmoid
- DenseResNet + ReLU

**Hint: Для отображения шумных величин можно воспользоваться оконным сглаживанием**

**Hint: Вам может пригодиться логарифмический масштаб по y**

In [ ]:
deep_model = <your code>

opt = torch.optim.SGD(deep_model.parameters(), lr=0.01)
train_model(deep_model, opt, train_loader, val_loader)

Реализуйте ResNet каким-либо способом.

Резнет собран из блоков вида $y = x + F(x)$, где $F$ -- это набор "обычных" слоев.

**(0.2 балла)** Проведите эксперименты с Dense ResNet ReLU и Dense ResNet Sigmoid:

In [ ]:
resnet = <your code>

opt = torch.optim.SGD(resnet.parameters(), lr=0.01)
train_model(resnet, opt, train_loader, val_loader)

## Сверточные сети

Мы рассмотрим сверточные сети на примере FashionMNIST.

В случае картинок, обычно работают с входными тензорами размера `[batch_size, channels, height, widht]` (такой порядок осей называется channels-first или NCHW).

Сверточные сети обычно собираются из последовательности слоев:

### Convolution
https://pytorch.org/docs/stable/nn.html#convolution-layers

По тензору бежит скользящее окно и в нем вычисляется свертка с ядром.
Обычно говорят о пространственных размерах сверток, например 1x1 или 3x3  свертки, подразумевая, что ядра имеют размер `[1,1,ch]` или `[3,3,ch]`.

Сейчас часто используются чуть более сложные варианты сверток: 
- dilated (atrous, дырявые), 
- depth-wise
- pointwise
- separable
- group


### Pooling
https://pytorch.org/docs/stable/nn.html#pooling-layers

Действуют аналогично свертках, но не имеют весов, а в бегущем окне вычисляется какая-нибудь функция, например max или mean.


### Global pooling (Adaptive Pooling)
https://pytorch.org/docs/stable/nn.html#adaptivemaxpool1d

Глобальные пулинги (в pytorch они называются адаптивными) убирают пространственные размерности, превращая `[bs, ch, h, w]` в `[bs, ch, 1, 1]`.



### Heads and body

Удобно выделять в сверточных сетях две части: полносверточную (body, feature extractor, тушка) и классификатор (head, голова).

Классификатор обычно состоит из полносвязных слоев (и где-то может обозначаться как MLP, MLP-head), и требует фиксированного размера тензоров (batch_size может варьироваться, но остальные размерности фиксированы).

Полносверточная часть обычно может работать на входах произвольных размеров (не меньше минимального).


Чтобы объединить эти две части используют какую-нибудь из операций: **Flatten** или **Global Pooling**.

#### Задание 1 (0.2 балла)

Реализуйте сверточную сеть, *2x(Conv+ReLU+MaxPooling) Conv + Relu + Flatten + (Dense + Relu + Dense)*.

Точность на валидации должна быть больше 0.9

Количество каналов и размеры фильтров выбирайте по желанию, дефолтный вариант 32-64-128 (3х3).

**Hint: Для последовательности слоев без skip-connections удобно пользоваться оберткой `nn.Sequential`.**

**NB: это упражнение стоит делать на GPU**

In [ ]:
cnn = <your code>
# В качестве быстрой проверки корректности попробуем прогнать через сеть тензор нужного размера
# [bs, ch, h, w]
x = torch.zeros([4, 1, 28, 28])
model(x).shape

In [ ]:
# Теперь сеть ожидает на вход картинки, так что достаточно просто преобразовать PIL.Image в Torch.Tensor
transform = transforms.Compose([
    transforms.ToTensor()
])
# имеет смысл добавить нормирование картинок

train_dataset = FashionMNIST("./tmp", train=True, download=True, transform=transform)
val_dataset = FashionMNIST("./tmp", train=False, download=True, transform=transform)
train_loader = DataLoader(train_dataset, shuffle=True, batch_size=32)
val_loader = DataLoader(val_dataset, shuffle=False, batch_size=32)

opt = torch.optim.SGD(cnn.parameters(), lr=0.01)
train_model(cnn, opt, train_loader, val_loader)